In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

# Lab 9: Climate data, CO2 and `xarray`

## Due Date

This assignment is due on **October 25th, 2022, at 11:59PM PDT**.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

This lab accompanies a lecture for Berkeley's Data 100 that covers the fundamental physical mechanisms behind global warming and analyzes CO2 and ocean temperature data.

Authors: Fernando Pérez and [Dr. Chelle Gentemann](https://cgentemann.github.io).

**Collaborators:** *list names here*

In [ ]:
# Run this cell to set up your notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from pathlib import Path

# Small style adjustments for more readable plots
plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

## Loading the Mauna Loa CO2 data

We start by loading the same dataset we used during the lecture, containing CO2 measurements in Mauna Loa, Hawaii.

In [ ]:
# Set up to load data either from the shared directory that has the complete set,
# or from the local small copy available in the repo

DATA_DIR = Path('./data')

# If you want to run this on the Berkeley data hub, where we have a larger version of the data,
# uncomment the below. But do not do that for submitting the Lab, as the version that will
# run on the grader needs to use the path above and only has enough data to grade the Lab.
#DATA_DIR = Path.home()/Path('shared/climate-data')

Now we start by loading the CO2 data from a CSV file:

In [ ]:
co2_file = DATA_DIR / "monthly_in_situ_co2_mlo_cleaned.csv"
data = pd.read_csv(co2_file)
data.head()

In [ ]:
# get rid of missing values that are set to -99.99 and then drop from the dataset
data = pd.read_csv(co2_file, na_values=-99.99).dropna()
data.head()

In [ ]:
# We did this in lecture, as a reminder this is what the data looks like:
plt.plot("fraction_date", "c02", data=data)
plt.plot("fraction_date", "data_adjusted_seasonally_fit", data=data);

# Exploring the annual anomaly

Notice how in the plot, there seems to be oscilliations within each year (changing by the months/seasons), but the overall trend of these oscillations appears to be going upward. We are going to try to understand the annual variability on top of the underlying growing trend, and see whether that variability within a given year is itself changing over time or not.

The figure above shows an annual cycle, alongside with perhaps some variability in it. As we saw in lecture (recall the super computer movie in talk with cities release of co2 strongest in winter), plants take up CO2 in (northern) spring/summer then release in fall/winter --- so the release is getting stronger. 

The annual cycles look a bit like waves, and recall that the amplitude of a wave is the height of its peak. Let's try to estimate the increase in amplitude of annual cycle over the years.

<!-- why would using a rolling mean make it look like the cycle is getting larger - both summer/winter rather than just winter? with this data can you figure out if summer or winter causing increase? -->

In the lecture notebook, we created the following figure, based on a quick and simple groupby operation and removal of the annual mean:

In [ ]:
# calculate the annual cycle using groupby
annual = data.groupby(data.month).mean()
# calculate the anomaly
anomaly = annual - annual.mean()

# plot
fig, ax = plt.subplots()

ax.plot("fraction_date", "data_filled", "r.", data=data)
ax.plot("fraction_date", "data_adjusted_seasonally_fit", data=data)
ax.set_xlabel("Year")
ax.set_ylabel("CO2 fraction in dry air (ppm)")
ax.set_title("Monthly Mean CO2")
ax.grid(False)

axin1 = ax.inset_axes([0.1, 0.5, 0.35, 0.35])
axin1.plot(anomaly.c02, "b")
axin1.plot(anomaly.c02, "r.")
axin1.set_title("Seasonal Anomaly");

But if you look closely, that figure isn't quite the same as the one shown in slide 9 of the lecture:

<img src="annual-anomaly-orig.png" width="700px" />

We'll start by trying to understand the monthly data. The following shows us the data for all the years, by month:

In [ ]:
plt.scatter('month', 'c02', data=data);

Unfortunately it's hard to see what's actually going on here.

### Question 1

Recreate the following figure, that shows the monthly cycle for all the years in the dataset:

<img src="monthly-cycles-co2.png" width="700px" />

Hint: Use `sns.lineplot(...)`


In [ ]:
...

### Question 2

Next, in order to attempt to recreate the figure in the talk, we're going to find the monthly anomaly averaging across year. To find this anomaly, we will first be "detrending" our data: we will be taking our data for each year, and subtracting off the mean from that year to get a sense of the variability about the average for each year. After detrending, we'll average the monthly data across all years. In this question, you will be writing code to detrend the data. You should end up with the following data frame after writing your function and running the cell (only the first few months are shown, it should have data for all 12 months):

<img src="monthly-co2-anomaly-df.png" width="150px" />


In [ ]:
def detrend(f):
    detrended_xarr = ...
    ...
    return detrended_xarr

dy = data.groupby('year')
c02anomaly = dy.apply(detrend)
display(c02anomaly)
monthly_anomaly = c02anomaly.groupby('month').mean()[['c02']]
monthly_anomaly

In [ ]:
grader.check("q2")

### Question 3

Next, recreate the following figure, which is much closer to the one in the lecture:

<img src="annual-anomaly-new.png" width="700px" />

**Hints:** 
- in order to get that smooth curve, you'll need to use `from scipy.interpolate import CubicSpline`. You can find the documentation for `CubicSpline` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html). Cubic splines are a very useful method to interpolate data between points (i.e., to draw a function connecting multiple points so we get a better idea of what the function may look like in between points). As an extra hint on using CubicSpline, it's implementation is quite similar to any sci-kit learn function you've already used.
- The "Seasonal Anomaly" inset graph can be created with `ax.inset_axes(...)`. You can treat the `Axes` object returned like any other `Axes` object.
- Remember that you can set the tick labels with `ax.set_xticks(...)`.
- To get the data points plotted as red dots, when you use `ax.plot(...)`, pass in `"r."` as the argument immediately following any data you pass in. The `"r"` indicates the color, while the `"."` indicates how the data will be plotted (if you don't put the `"."`, it will be a line instead).


In [ ]:
from scipy.interpolate import CubicSpline

fig, ax = plt.subplots()

...

# Exploring Snow with ERA 5

Now, we turn our attention to the global ERA 5 dataset and explore some other questions - it's a very rich and interesting dataset, and the lecture only scratched its surfac!

We are going today to focus on just one more bit: we'll take a look at the snow accumulation data for the northern and southern hemispheres.

But you should see this as an invitation to keep learning from these data! Think of looking at other variables in the dataset. Is there annual cycle? trend? Some of the data might look very different than the air temperature - eg. precipitation which is either 0 or +. Can you use PDFs to look at changes in distributions over a region? at a point? Or talk about the data a little & what you understand it is measuring? Are any of the data variables related to each other? Can you plot correlations between data?

In [ ]:
import xarray as xr

monthly_2deg_path = DATA_DIR / "era5_snow_monthly_2deg_aws_v20210920.nc"

# The Berkeley hub has a version with not only snow data but many more variables.
# You can explore the full dataset by uncommenting the following if you are on the
# campus hub. Do NOT do that for your lab submission, as that data is NOT available
# on Gradescope.

# Use this version only when running on the hub:
# monthly_2deg_path = Path("~/shared/climate-data").expanduser() / "era5_monthly_2deg_aws_v20210920.nc"

ds = xr.open_dataset(monthly_2deg_path)
ds

Consider a standard `pandas` DataFrame, where each observation has an integer index and each column contains variable information about that observation. For physical data, we often have data that is collected at points at a particular location on Earth at a particular time. We can think of this data as being indexed by three-dimensions: longitude, latitude, and time. This part of the lab uses a package called `xarray`, a package with a very similar functionality as `pandas`, but is much more versatile in that it allows a user to work easily with such data that may have multidimensional indexing.

What we just made above was called an `xarray.Dataset` object, a collection of a bunch of `xarray.DataArray` objects. Intuitively, a single `xarray.DataArray` object contains measurements of a single variable indexed across different locations in space and time, and multiple `xarray.DataArray` objects for different variables can be grouped into a single `xarray.Dataset` for easier bookkeeping/organization.

`xarray` also has a really neat interactive feature when you print out a `Dataset` or a `DataArray`: you can click to see the different Dimensions of the dataset, the Coordinates that data points are collected at (in space, given by latitude/longitude combinations, and time), and variable information collected at each point in space and time (Data variables). 

Feel free to click around on the above output! When cliking the rightmost button for `time`, `latitude`, and `longitude` coordinates, we see that each of these are actually just NumPy arrays, giving us the latitudes/longitudes as well as times that our observations were collected! The Data variables section always tells you what variables you are collecting information about at each of these latitude-longitude-time coordinates. Here, we are collecting information about snow density, at different points in space across time.

`xarray` has a rich documentation, so you can always check there if you have questions. However, when in doubt, using a `Pandas` type implementation to get something done might work since the packages are so similar. For example, if I wanted all of the unique longitudes in my dataset, I could just do the following, which spits out a DataArray of my longitudes.

In [ ]:
ds['longitude']

We can imagine we have snow density measurements across three axes: latitude, longitude, and time. If we take all measurements for a particular time (let's say, year), we would have data on snow measurements for each latitude and longitudes for that particular year, a snapshot in time of the worldwide snow density. However, let's suppose we wish to get the snow density measurements for each latitude and longitude, averaged across all the times in the dataset. We are essentially then aggregating over time, such that each latitude/longitude location now only has one snow density measurement: the average over all the years for that location.

In [ ]:
mean_map = ds.mean("time")  # takes the mean across all variables in ds
mean_map.snow_density.plot();

We can also extract the `snow_density` information from our `xarray.Dataset` to return a `xarray.DataArray`, in the same way we might take a column from a `pandas.DataFrame`. Once again, the output is interactive, so it's really helpful to understand what exactly is contained in this dataset. Provided below is the snow density measurements are given in the from of numpy arrays stacked on top of each other. 

Consider for a second why this data structure makes sense: stacked numpy arrays are three-dimensional NumPy arrays, or collections of numbers indexed by three numbers as opposed to just two in a standard array or one as in a list. Snow density measurements are given across three dimensions (time, latitude, longitude), so the numpy array with this information should be three dimensional. 

Once again, `xarray` tells us what each axis of this numpy array means (time, latitude, and longitude), and also gives us some information in the Attributes section, such as the units `snow_density` is measured in.

In [ ]:
snow = ds.snow_density
snow

### Question 4

Let's look at the snow density measurements across the globe in February and August, respectively. Februrary is considered the peak of winter in the Northern Hemisphere and the peak of summer in the Southern Hemisphere, and vice versa for August.

Recreate the following figure, along with a corresponding one for August 1980. 

*Hint:* using the `snow` variable above, which is an `xarray.Dataset` object of snow density measurements across time and location, how can you extract all of the measurements for February 1980? for August 1980? See [this page](https://docs.xarray.dev/en/stable/user-guide/indexing.html) from the `xarray` documentation about how to index into these objects (try to avoid the methods where you use integer indexing, the beauty of `xarray` is it's label indexing!) For plotting, try to see if there are any similarities with plotting in `pandas` DataFrames.

<img src="snow-distribution-1980-02.png" width="500px" />


In [ ]:
...

In [ ]:
...

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)